# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries}
import pandas as pd 
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle
import os


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alan.jeffares/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alan.jeffares/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alan.jeffares/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('tweets', con=engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [58]:
a = 150
for i in range(a, a+40):
    print(X[i])

I am in Carrefour on a street near EDH ( Electricity of Haiti? ) Help us please
Everyone, do not forget the people of Merger, we don't have any water, we are hungry. If it rains, we will all die
Digicel, I have a problem, my family doesn't know where I am, I can't call, please help
Please. Send humanitarian aid so we can survive. We are in Delmas 40B village Musso on the golf course. Thank you
good evening we ? carrefour feuill ???we are dying of hunger and ther is no water pleas help us
the people in the village of god need food and water so that they can stay. It is isn the university of Kiskeya ( in the area of delams 35 right of the main delmas road ). pleas respond
Can someone who has a visa travel out. What can he do?
there are alot of victimes that have arrived ath the Marchand Desalin and nan Ranboto hospital. they do not have enough supplys or specialist ( surgeons ) i would like this looked into
help us at delmas 65 orchidee street, we are thirsty. thanks
we want people to kn

### 2. Write a tokenization function to process your text data

In [112]:
print(X[3])
tokenize(X[3])

# tokens = word_tokenize(X[0])
# lemmatizer = WordNetLemmatizer()

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.


['un',
 'report',
 'leogane',
 '80-90',
 'destroyed',
 '.',
 'only',
 'hospital',
 'st.',
 'croix',
 'functioning',
 '.',
 'needs',
 'supply',
 'desperately',
 '.']

In [65]:
def tokenize(text):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [66]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [170]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', (MLPClassifier()))
    ])

# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ])),

#             ('starting_verb', StartingVerbExtractor())
#         ])),

#         ('clf', DecisionTreeClassifier())
#     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [171]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.1)
pipeline.fit(X_train, Y_train)

/anaconda3/envs/disaster_response_env/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [173]:
Y_pred = pipeline.predict(X_test)
Y_pred = pd.DataFrame(Y_pred, columns=Y_train.columns)

In [147]:
# pipeline.get_params()

In [174]:
results = pd.DataFrame()
for column_name in Y_pred.columns:
    col_report = classification_report(y_true=Y_test[[column_name]], y_pred=Y_pred[[column_name]], output_dict=True)
    accuracy = col_report['accuracy']
    precision = col_report['macro avg']['precision']
    recall = col_report['macro avg']['recall']
    results[column_name] = [accuracy, precision, recall]
results.index = ['accuracy', 'precision', 'recall']
results.mean(axis=1)

/anaconda3/envs/disaster_response_env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy     0.937172
precision    0.721678
recall       0.578209
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [150]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', DecisionTreeClassifier())
    ])

parameters = [
    {
        'features__text_pipeline__vect__max_df': (0.5, 1.0),
        'features__text_pipeline__vect__min_df': (1, 0.01),
        'features__text_pipeline__vect__max_features': (None, 5000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf': (DecisionTreeClassifier(min_samples_split=3),),
        'clf__max_depth': (None, 4)
    }, {
        'features__text_pipeline__vect__max_df': (0.5, 1.0),
        'features__text_pipeline__vect__min_df': (1, 0.01),
        'features__text_pipeline__vect__max_features': (None, 5000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf': (MultiOutputClassifier(LinearSVC(multi_class='ovr')),)
    }, {
        'features__text_pipeline__vect__max_df': (0.5, 1.0),
        'features__text_pipeline__vect__min_df': (1, 0.01),
        'features__text_pipeline__vect__max_features': (None, 5000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf': (MLPClassifier(),),
        'clf__hidden_layer_sizes': ((100, 10), (50,), (50, 10))
    }
]

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=4, verbose=10)

In [138]:
X_test.shape

(20972,)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [151]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   59.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done 288 out of 288 | elapsed: 22.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [180]:
Y_pred = cv.predict(X_test)
Y_pred = pd.DataFrame(Y_pred, columns=Y_train.columns)

In [181]:
results = pd.DataFrame()
for column_name in Y_pred.columns:
    col_report = classification_report(y_true=Y_test[[column_name]], y_pred=Y_pred[[column_name]], output_dict=True)
    accuracy = col_report['accuracy']
    precision = col_report['macro avg']['precision']
    recall = col_report['macro avg']['recall']
    results[column_name] = [accuracy, precision, recall]
results.index = ['accuracy', 'precision', 'recall']
results.mean(axis=1)

accuracy     0.948745
precision    0.891526
recall       0.626660
dtype: float64

In [11]:
add_header = not os.path.isfile('Model_results.csv')


True

In [1]:
model_name = type(cv.best_params_['clf']).__name__
avg_accuracy = results.mean(axis=1)['accuracy']
avg_precision = results.mean(axis=1)['precision']
avg_recall = results.mean(axis=1)['recall']
params = cv.best_params_

NameError: name 'cv' is not defined

In [218]:
stored_results = pd.DataFrame({'Model': [model_name], 'Accuracy': [avg_accuracy], 'Precision': [avg_precision], 
                               'Recall': [avg_recall], 'Parameters': [params]})

add_header = not os.path.isfile('Model_results.csv')

with open('Model_results.csv', 'a') as f:
    stored_results.to_csv(f, header=add_header, index=False)

In [219]:
stored_results

,Model,Accuracy,Precision,Recall,Parameters
27/10/19 15:02,MultiOutputClassifier,0.948745,0.891526,0.62666,{'clf': MultiOutputClassifier(estimator=Linear...


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [169]:
rank = pd.Series(cv.cv_results_['rank_test_score'])
clf = pd.Series(cv.cv_results_['param_clf'])
results = pd.DataFrame({'rank':rank, 'clf': clf})
results.sort_values('rank')

,rank,clf
40,1,MultiOutputClassifier(estimator=LinearSVC(C=1....
34,2,MultiOutputClassifier(estimator=LinearSVC(C=1....
46,2,MultiOutputClassifier(estimator=LinearSVC(C=1....
42,4,MultiOutputClassifier(estimator=LinearSVC(C=1....
44,5,MultiOutputClassifier(estimator=LinearSVC(C=1....
...,...,...
92,92,"MLPClassifier(activation='relu', alpha=0.0001,..."
54,93,"MLPClassifier(activation='relu', alpha=0.0001,..."
52,94,"MLPClassifier(activation='relu', alpha=0.0001,..."
84,95,"MLPClassifier(activation='relu', alpha=0.0001,..."


In [178]:
with open('model.pkl','wb') as f:
    pickle.dump(cv, f)

### 9. Export your model as a pickle file

In [175]:
# cv.best_estimator_.get_params()

In [17]:
import joblib

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [20]:
model = pickle.load(open("models/classifier.pkl", 'rb'))

ModuleNotFoundError: No module named 'model_utils'